Step 1- collect the extended streaming history from the 4 json files. They cover listening history from 2016 to 2023 
The listening history is not homogeneusly distributed throgout hte years we will also look at that.


In [16]:
import pandas as pd
import json as json
import os as os
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import requests  as requests
from requests import get
from requests import post
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials
import time
import base64

In [17]:
# List to store data from each file
data_list = []

# Replace 'path/to/json/files' with the actual path to your JSON files
json_files_path = ["C:/Users/Baran/Documents/GitHub/DataScienceProject/Data/SpotifyExtendedStreamingHistory/Streaming_History_Audio_2016-2021_0.json",
             "C:/Users/Baran/Documents/GitHub/DataScienceProject/Data/SpotifyExtendedStreamingHistory/Streaming_History_Audio_2021-2022_1.json",
             "C:/Users/Baran/Documents/GitHub/DataScienceProject/Data/SpotifyExtendedStreamingHistory/Streaming_History_Audio_2022-2023_2.json",
             "C:/Users/Baran/Documents/GitHub/DataScienceProject/Data/SpotifyExtendedStreamingHistory/Streaming_History_Audio_2023_3.json"]

# Iterate through each JSON file in the directory
for filename in json_files_path:
        print(filename)
        with open(filename, 'r', encoding='utf-8') as file:
        # Load each line as a separate JSON object
            for line in file:
                data = json.loads(line)
                data_list.append(data)

# Create a Pandas DataFrame from the list of JSON data
df = pd.DataFrame(data_list)

# Display the DataFrame
combined_row = df.apply(lambda x: ','.join(x.astype(str)), axis=0)

# Create a new DataFrame with the combined row
combined_df = pd.DataFrame([combined_row.tolist()], columns=df.columns)



display(combined_df.info())



C:/Users/Baran/Documents/GitHub/DataScienceProject/Data/SpotifyExtendedStreamingHistory/Streaming_History_Audio_2016-2021_0.json
C:/Users/Baran/Documents/GitHub/DataScienceProject/Data/SpotifyExtendedStreamingHistory/Streaming_History_Audio_2021-2022_1.json
C:/Users/Baran/Documents/GitHub/DataScienceProject/Data/SpotifyExtendedStreamingHistory/Streaming_History_Audio_2022-2023_2.json
C:/Users/Baran/Documents/GitHub/DataScienceProject/Data/SpotifyExtendedStreamingHistory/Streaming_History_Audio_2023_3.json
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Columns: 16880 entries, 0 to 16879
dtypes: object(16880)
memory usage: 132.0+ KB


None

In [18]:
melted_df = pd.melt(df)
melted_df = melted_df.drop(columns=['variable'])
melted_df = melted_df.dropna()
melted_df = melted_df.reset_index(drop=True)
melted_df['value'] = melted_df['value'].astype(str)


display(melted_df)
display(melted_df.info())

,value
0,"{'ts': '2016-03-19T20:15:16Z', 'username': 'ba..."
1,"{'ts': '2021-07-24T09:39:17Z', 'username': 'ba..."
2,"{'ts': '2022-06-30T06:00:17Z', 'username': 'ba..."
3,"{'ts': '2023-02-11T19:07:31Z', 'username': 'ba..."
4,"{'ts': '2016-03-19T20:15:54Z', 'username': 'ba..."
...,...
65940,"{'ts': '2023-02-11T19:02:35Z', 'username': 'ba..."
65941,"{'ts': '2023-02-11T19:02:35Z', 'username': 'ba..."
65942,"{'ts': '2023-02-11T19:02:44Z', 'username': 'ba..."
65943,"{'ts': '2023-02-11T19:07:26Z', 'username': 'ba..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65945 entries, 0 to 65944
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   value   65945 non-null  object
dtypes: object(1)
memory usage: 515.3+ KB


None

In [19]:
# Define the column list
column_list = ["ts","username","platform","ms_played","conn_country","ip_addr_decrypted",\
   "user_agent_decrypted","master_metadata_track_name","master_metadata_album_artist_name",\
    "master_metadata_album_album_name","spotify_track_uri","episode_name","episode_show_name",\
    "spotify_episode_uri","reason_start","reason_end","shuffle","skipped",'"offline',\
    'offline_timestamp','incognito_mode']

# Define a custom parsing function
def parse_data(row):
    data_dict = eval(row)  # Assuming your data is a string representation of a dictionary
    return pd.Series([data_dict.get(col, None) for col in column_list])

# Apply the parsing function to create new columns
parsed_df = melted_df['value'].apply(parse_data)

# Concatenate the parsed DataFrame with the original melted DataFrame
result_df = pd.concat([melted_df, parsed_df], axis=1)

# Drop the old 'variable' and 'value' columns
result_df = result_df.drop(columns=['value'])

# Rename the new columns based on the column list
result_df.columns = column_list

display(result_df)
display(result_df.info())

,ts,username,platform,ms_played,conn_country,ip_addr_decrypted,user_agent_decrypted,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,...,episode_name,episode_show_name,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,"""offline",offline_timestamp,incognito_mode
0,2016-03-19T20:15:16Z,baanozcan03,Windows 10 (10.0.10586; x64),295502,TR,78.172.125.37,unknown,Hello,Adele,Hello,...,None,None,None,clickrow,trackdone,False,None,None,0,False
1,2021-07-24T09:39:17Z,baanozcan03,"iOS 14.4.2 (iPhone12,1)",181379,TR,212.253.204.122,unknown,Kafamda Kentsel Dönüşümler,İkiye On Kala,Kafamda Kentsel Dönüşümler,...,None,None,None,playbtn,trackdone,False,None,None,1627119376479,False
2,2022-06-30T06:00:17Z,baanozcan03,"iOS 15.5 (iPhone12,1)",2645,TR,5.47.73.166,unknown,Empty Bed,Motorama,Alps,...,None,None,None,fwdbtn,fwdbtn,False,None,None,1656568814023,False
3,2023-02-11T19:07:31Z,baanozcan03,ios,1230,TR,37.154.245.49,unknown,Free Bird,Lynyrd Skynyrd,Pronounced' Leh-'Nerd 'Skin-'Nerd,...,None,None,None,fwdbtn,fwdbtn,True,True,None,1676142450,False
4,2016-03-19T20:15:54Z,baanozcan03,Windows 10 (10.0.10586; x64),31880,TR,78.172.125.37,unknown,Cheerleader (Felix Jaehn Remix) - Radio Edit,OMI,Cheerleader,...,None,None,None,clickrow,endplay,False,None,None,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65940,2023-02-11T19:02:35Z,baanozcan03,ios,998,TR,37.154.245.49,unknown,Don't Stay,Linkin Park,Meteora,...,None,None,None,fwdbtn,fwdbtn,True,True,None,1676142155,False
65941,2023-02-11T19:02:35Z,baanozcan03,ios,133882,TR,37.154.245.49,unknown,Down Under,Men At Work,Business As Usual,...,None,None,None,trackdone,fwdbtn,True,True,None,1676142020,False
65942,2023-02-11T19:02:44Z,baanozcan03,ios,8591,TR,37.154.245.49,unknown,Buried Alive,Avenged Sevenfold,Nightmare,...,None,None,None,fwdbtn,endplay,True,True,None,1676142156,False
65943,2023-02-11T19:07:26Z,baanozcan03,ios,282800,TR,37.154.245.49,unknown,Single Handed Sailor,Dire Straits,Communiqué,...,None,None,None,clickrow,trackdone,True,False,None,1676142165,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65945 entries, 0 to 65944
Data columns (total 21 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   ts                                 65945 non-null  object
 1   username                           65945 non-null  object
 2   platform                           65945 non-null  object
 3   ms_played                          65945 non-null  int64 
 4   conn_country                       65945 non-null  object
 5   ip_addr_decrypted                  64692 non-null  object
 6   user_agent_decrypted               64692 non-null  object
 7   master_metadata_track_name         64538 non-null  object
 8   master_metadata_album_artist_name  64538 non-null  object
 9   master_metadata_album_album_name   64538 non-null  object
 10  spotify_track_uri                  64538 non-null  object
 11  episode_name                       1392 non-null   object
 12  epis

None

In [20]:
#time to make columns a bit more pleaseant to look at

result_df['ts'] = pd.to_datetime(result_df['ts'], errors='coerce')
nan_counts = result_df.isna().sum()
print(nan_counts)
display(result_df)

ts                                       0
username                                 0
platform                                 0
ms_played                                0
conn_country                             0
ip_addr_decrypted                     1253
user_agent_decrypted                  1253
master_metadata_track_name            1407
master_metadata_album_artist_name     1407
master_metadata_album_album_name      1407
spotify_track_uri                     1407
episode_name                         64553
episode_show_name                    64553
spotify_episode_uri                  64553
reason_start                             0
reason_end                            1253
shuffle                                  0
skipped                              42006
"offline                             65945
offline_timestamp                        0
incognito_mode                           0
dtype: int64


,ts,username,platform,ms_played,conn_country,ip_addr_decrypted,user_agent_decrypted,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,...,episode_name,episode_show_name,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,"""offline",offline_timestamp,incognito_mode
0,2016-03-19 20:15:16+00:00,baanozcan03,Windows 10 (10.0.10586; x64),295502,TR,78.172.125.37,unknown,Hello,Adele,Hello,...,None,None,None,clickrow,trackdone,False,None,None,0,False
1,2021-07-24 09:39:17+00:00,baanozcan03,"iOS 14.4.2 (iPhone12,1)",181379,TR,212.253.204.122,unknown,Kafamda Kentsel Dönüşümler,İkiye On Kala,Kafamda Kentsel Dönüşümler,...,None,None,None,playbtn,trackdone,False,None,None,1627119376479,False
2,2022-06-30 06:00:17+00:00,baanozcan03,"iOS 15.5 (iPhone12,1)",2645,TR,5.47.73.166,unknown,Empty Bed,Motorama,Alps,...,None,None,None,fwdbtn,fwdbtn,False,None,None,1656568814023,False
3,2023-02-11 19:07:31+00:00,baanozcan03,ios,1230,TR,37.154.245.49,unknown,Free Bird,Lynyrd Skynyrd,Pronounced' Leh-'Nerd 'Skin-'Nerd,...,None,None,None,fwdbtn,fwdbtn,True,True,None,1676142450,False
4,2016-03-19 20:15:54+00:00,baanozcan03,Windows 10 (10.0.10586; x64),31880,TR,78.172.125.37,unknown,Cheerleader (Felix Jaehn Remix) - Radio Edit,OMI,Cheerleader,...,None,None,None,clickrow,endplay,False,None,None,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65940,2023-02-11 19:02:35+00:00,baanozcan03,ios,998,TR,37.154.245.49,unknown,Don't Stay,Linkin Park,Meteora,...,None,None,None,fwdbtn,fwdbtn,True,True,None,1676142155,False
65941,2023-02-11 19:02:35+00:00,baanozcan03,ios,133882,TR,37.154.245.49,unknown,Down Under,Men At Work,Business As Usual,...,None,None,None,trackdone,fwdbtn,True,True,None,1676142020,False
65942,2023-02-11 19:02:44+00:00,baanozcan03,ios,8591,TR,37.154.245.49,unknown,Buried Alive,Avenged Sevenfold,Nightmare,...,None,None,None,fwdbtn,endplay,True,True,None,1676142156,False
65943,2023-02-11 19:07:26+00:00,baanozcan03,ios,282800,TR,37.154.245.49,unknown,Single Handed Sailor,Dire Straits,Communiqué,...,None,None,None,clickrow,trackdone,True,False,None,1676142165,False


In [23]:
#Seperate podcasts and music
podcast_df = result_df[result_df['episode_name'].notna()]
music_df = result_df[result_df['episode_name'].isna()]
podcast_df = podcast_df.sort_values(by='ts')
music_df = music_df.sort_values(by='ts')

#drop rows that dont have corresponding URI and show the dropped row count
podcast_df = podcast_df.dropna(subset=['spotify_episode_uri'])
music_df = music_df.dropna(subset=['spotify_track_uri'])



##lets conver the ms_played to seconds with 2 decimal places
podcast_df['ms_played'] = podcast_df['ms_played']/1000
podcast_df['ms_played'] = podcast_df['ms_played'].round(2)
music_df['ms_played'] = music_df['ms_played']/1000
music_df['ms_played'] = music_df['ms_played'].round(2)
#rename it to seconds_played
podcast_df = podcast_df.rename(columns={'ms_played': 'seconds_played'})
music_df = music_df.rename(columns={'ms_played': 'seconds_played'})




# Drop columns where all values are NaN
podcast_df = podcast_df.dropna(axis=1, how='all')
music_df = music_df.dropna(axis=1, how='all')


##lets create a season column 
# Define a function to determine the season
def get_season(month):
    if 3 <= month <= 5:
        return 'Spring'
    elif 6 <= month <= 8:
        return 'Summer'
    elif 9 <= month <= 11:
        return 'Fall'
    else:
        return 'Winter'

# Create a new 'season' column based on the 'ts' column
podcast_df['season'] = podcast_df['ts'].dt.month.apply(get_season)
music_df['season'] = music_df['ts'].dt.month.apply(get_season)

In [ ]:
#time to get song IDs
song_IDs = music_df['spotify_track_uri'].str.split(':').str[-1].unique().tolist()
print(song_IDs)

backup_song_IDs = song_IDs.copy()

client_id = ''
client_secret = ''



def get_token():
    authotization_str = client_id + ':' + client_secret
    authotization_bytes = authotization_str.encode('utf-8')
    base64_encoded = str(base64.b64encode(authotization_bytes), 'utf-8')

    url = 'https://accounts.spotify.com/api/token'
    headers = {'Authorization': 'Basic ' + base64_encoded, 'Content-Type': 'application/x-www-form-urlencoded'}
    dat = {'grant_type': 'client_credentials'}
    req = post(url, headers=headers, data=data)
    json_result = json.loads(req.content)
    token = json_result['access_token']
    return token

auth_header = {'Authorization': 'Bearer ' + get_token()}
print(auth_header)


def get_track_features(track_id, auth_header):
    url = 'https://api.spotify.com/v1/audio-features/' + track_id
    req = get(url, headers=auth_header)
    json_result = json.loads(req.content)
    return json_result

# Replace 'your_results' with the desired base file name
base_file_name = 'my_results'

# Set the chunk size
chunk_size = 500

# Calculate the number of chunks
num_chunks = (len(song_IDs) + chunk_size - 1) // chunk_size
'''
# Iterate through each chunk
for chunk_number in range(10,num_chunks):
    # Determine the start and end indices for the current chunk
    start_index = chunk_number * chunk_size
    end_index = (chunk_number + 1) * chunk_size

    # Get the track IDs for the current chunk
    current_chunk_ids = song_IDs[start_index:end_index]

    # Create the file name for the current chunk
    file_name = f'{base_file_name}_chunk_{chunk_number}.txt'

   
    # Open the file in write mode ('w')
    with open(file_name, 'w') as file:
        # Iterate through each response and write it to the file
        for id in current_chunk_ids:
            print(current_chunk_ids.index(id))
            time.sleep(0.4)
            response = get_track_features(id, auth_header)
            print(response)
            file.write(str(response) + '\n')  # Adjust this based on your data format
'''

In [25]:
# Replace 'CS210Project' with your actual project folder
project_folder = 'C:/Users/Baran/Documents/GitHub/DataScienceProject/Data'

# Specify the base file name
base_file_name = 'my_results_chunk_'

# Specify the file extension
file_extension = 'txt'
combined_list  = []

# Iterate over chunk numbers from 0 to 10
for chunk_number in range(11):  # This will include 0 to 10
    # Create the file name for the current chunk
    file_name = f'{base_file_name}{chunk_number}.{file_extension}'

    # Create the complete file path using os.path.join
    file_path = os.path.join(project_folder, file_name)
    file_path = file_path.replace('\\', '/')  # Replace backslashes with forward slashes

    # Read the text file line by line, parse each line as JSON, and append to DataFrame
    with open(file_path, 'r') as file:
        for line in file:
            combined_list.append(line)

# Your list of column names
column_list = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature']

# Convert the list of strings to a pandas DataFrame
track_properties_df = pd.DataFrame([eval(line) for line in combined_list], columns=column_list)

# Display the DataFrame
display(track_properties_df)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.471,0.431,5,-6.129,0,0.0342,0.329000,0.000000,0.0854,0.289,157.980,audio_features,1MDoll6jK4rrk2BcFRP5i7,spotify:track:1MDoll6jK4rrk2BcFRP5i7,https://api.spotify.com/v1/tracks/1MDoll6jK4rr...,https://api.spotify.com/v1/audio-analysis/1MDo...,295502,4
1,0.827,0.576,4,-7.774,1,0.0372,0.184000,0.029000,0.0656,0.410,118.040,audio_features,6fQcqEcXfnY0L6VwS0c3gT,spotify:track:6fQcqEcXfnY0L6VwS0c3gT,https://api.spotify.com/v1/tracks/6fQcqEcXfnY0...,https://api.spotify.com/v1/audio-analysis/6fQc...,180021,4
2,0.540,0.766,4,-5.148,0,0.0700,0.000248,0.000000,0.6410,0.176,98.932,audio_features,5NUlOzzMdoIABFTnhizCQa,spotify:track:5NUlOzzMdoIABFTnhizCQa,https://api.spotify.com/v1/tracks/5NUlOzzMdoIA...,https://api.spotify.com/v1/audio-analysis/5NUl...,224400,4
3,0.702,0.770,9,-5.416,0,0.0482,0.124000,0.000000,0.1030,0.593,107.951,audio_features,6RsWqX8zABZLhZydXxEFOm,spotify:track:6RsWqX8zABZLhZydXxEFOm,https://api.spotify.com/v1/tracks/6RsWqX8zABZL...,https://api.spotify.com/v1/audio-analysis/6RsW...,213520,4
4,0.593,0.748,0,-3.784,0,0.0461,0.151000,0.000000,0.3010,0.468,100.007,audio_features,0XXLwO7YHHTAK3OJkWQ37b,spotify:track:0XXLwO7YHHTAK3OJkWQ37b,https://api.spotify.com/v1/tracks/0XXLwO7YHHTA...,https://api.spotify.com/v1/audio-analysis/0XXL...,219987,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5470,0.374,0.528,7,-14.547,1,0.1900,0.104000,0.168000,0.1180,0.410,82.037,audio_features,1rOPLMaQP8yDdh1fPLIyXz,spotify:track:1rOPLMaQP8yDdh1fPLIyXz,https://api.spotify.com/v1/tracks/1rOPLMaQP8yD...,https://api.spotify.com/v1/audio-analysis/1rOP...,224211,4
5471,0.946,0.572,9,-6.688,1,0.1330,0.006610,0.000009,0.0711,0.297,124.965,audio_features,6cOk7Ry9kZmnNAzVscxwnw,spotify:track:6cOk7Ry9kZmnNAzVscxwnw,https://api.spotify.com/v1/tracks/6cOk7Ry9kZmn...,https://api.spotify.com/v1/audio-analysis/6cOk...,243833,4
5472,0.675,0.322,6,-12.126,0,0.1900,0.893000,0.000000,0.1630,0.395,79.059,audio_features,7pxVwm2jd4LZyT68E78123,spotify:track:7pxVwm2jd4LZyT68E78123,https://api.spotify.com/v1/tracks/7pxVwm2jd4LZ...,https://api.spotify.com/v1/audio-analysis/7pxV...,60797,4
5473,0.455,0.758,0,-11.361,1,0.0454,0.223000,0.000003,0.0986,0.540,141.304,audio_features,2APxH5XbOLDTmW9X5yi6Hw,spotify:track:2APxH5XbOLDTmW9X5yi6Hw,https://api.spotify.com/v1/tracks/2APxH5XbOLDT...,https://api.spotify.com/v1/audio-analysis/2APx...,294000,4


In [38]:
#lest merge the two dataframes of music and track properties if the track uri matches
# Sample DataFrames

# Merge the DataFrames if the 'uri' columns match
music_df = music_df.rename(columns={'spotify_track_uri': 'uri'})
merged_df = pd.merge(music_df, track_properties_df, on='uri', how='left')



#test to see if the merge worked
print(len(music_df.columns))
print(len(track_properties_df.columns))
print(len(merged_df.columns))
#print all instances of master_metadata_track_name == 'Isolation Years' as a test
display(merged_df.loc[merged_df['master_metadata_track_name'] == 'Isolation Years'])

18
18
35


,ts,username,platform,seconds_played,conn_country,ip_addr_decrypted,user_agent_decrypted,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
50545,2023-03-08 18:20:50+00:00,baanozcan03,ios,4.50,TR,159.20.68.5,unknown,Isolation Years,Opeth,Ghost Reveries,...,0.000347,0.102,0.11,53.297,audio_features,5HGzGElmhG7Fk2d0qh00Zh,https://api.spotify.com/v1/tracks/5HGzGElmhG7F...,https://api.spotify.com/v1/audio-analysis/5HGz...,231427,4
50547,2023-03-08 18:22:06+00:00,baanozcan03,ios,20.86,TR,159.20.68.5,unknown,Isolation Years,Opeth,Ghost Reveries,...,0.000347,0.102,0.11,53.297,audio_features,5HGzGElmhG7Fk2d0qh00Zh,https://api.spotify.com/v1/tracks/5HGzGElmhG7F...,https://api.spotify.com/v1/audio-analysis/5HGz...,231427,4
50576,2023-03-09 12:32:41+00:00,baanozcan03,ios,2.16,TR,159.20.68.5,unknown,Isolation Years,Opeth,Ghost Reveries,...,0.000347,0.102,0.11,53.297,audio_features,5HGzGElmhG7Fk2d0qh00Zh,https://api.spotify.com/v1/tracks/5HGzGElmhG7F...,https://api.spotify.com/v1/audio-analysis/5HGz...,231427,4
50578,2023-03-09 12:33:38+00:00,baanozcan03,ios,54.61,TR,159.20.68.5,unknown,Isolation Years,Opeth,Ghost Reveries,...,0.000347,0.102,0.11,53.297,audio_features,5HGzGElmhG7Fk2d0qh00Zh,https://api.spotify.com/v1/tracks/5HGzGElmhG7F...,https://api.spotify.com/v1/audio-analysis/5HGz...,231427,4
50627,2023-03-10 12:22:29+00:00,baanozcan03,ios,231.43,TR,94.235.148.44,unknown,Isolation Years,Opeth,Ghost Reveries,...,0.000347,0.102,0.11,53.297,audio_features,5HGzGElmhG7Fk2d0qh00Zh,https://api.spotify.com/v1/tracks/5HGzGElmhG7F...,https://api.spotify.com/v1/audio-analysis/5HGz...,231427,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62178,2023-09-29 15:42:12+00:00,baanozcan03,ios,111.46,TR,37.154.251.96,unknown,Isolation Years,Opeth,Ghost Reveries,...,0.000347,0.102,0.11,53.297,audio_features,5HGzGElmhG7Fk2d0qh00Zh,https://api.spotify.com/v1/tracks/5HGzGElmhG7F...,https://api.spotify.com/v1/audio-analysis/5HGz...,231427,4
63699,2023-11-12 07:53:36+00:00,baanozcan03,ios,231.43,TR,37.154.248.104,unknown,Isolation Years,Opeth,Ghost Reveries,...,0.000347,0.102,0.11,53.297,audio_features,5HGzGElmhG7Fk2d0qh00Zh,https://api.spotify.com/v1/tracks/5HGzGElmhG7F...,https://api.spotify.com/v1/audio-analysis/5HGz...,231427,4
63890,2023-11-22 07:54:21+00:00,baanozcan03,ios,192.05,TR,159.20.69.0,unknown,Isolation Years,Opeth,Ghost Reveries,...,0.000347,0.102,0.11,53.297,audio_features,5HGzGElmhG7Fk2d0qh00Zh,https://api.spotify.com/v1/tracks/5HGzGElmhG7F...,https://api.spotify.com/v1/audio-analysis/5HGz...,231427,4
64287,2023-12-15 18:07:35+00:00,baanozcan03,windows,95.71,TR,159.20.69.23,unknown,Isolation Years,Opeth,Ghost Reveries,...,0.000347,0.102,0.11,53.297,audio_features,5HGzGElmhG7Fk2d0qh00Zh,https://api.spotify.com/v1/tracks/5HGzGElmhG7F...,https://api.spotify.com/v1/audio-analysis/5HGz...,231427,4


In [ ]:
#the data is now ready to be explored